In [24]:
import bs4

### Goal Data Dictionary

In [25]:
file = open('../Downloads/Bounding_Box/git_annotation_2020_01_23_09_43_20.dump', 'r')

In [26]:
def create_goal_data(file):

    goal_data_dict = {}

    all_id, all_start, all_stop, soup = get_image_list(file)

    for id_index in range(len(all_id)):

        # From image
        start = all_start[id_index]

        # Upto image
        stop = all_stop[id_index]

        for img in range(start, stop+1):

            # Image content
            image = soup.find('image', id=img)

            if image:

                image_dict = image_evaluation(image)
                goal_data_dict[img] = image_dict
                
    return goal_data_dict

In [27]:
goal_data_dict = create_goal_data(file)
goal_data_dict

{0: {'box1': {'label': 'Automobile',
   'coordinates': [455.9, 496.3, 401.8, 401.8],
   'attributes': ['Truncation Fully Visible', 'Occlusion Fully Visible']},
  'box2': {'label': 'Automobile',
   'coordinates': [430.2, 474.02, 400.3, 400.3],
   'attributes': ['Truncation Fully Visible', 'Occlusion Fully Visible']},
  'box3': {'label': 'Heavy Truck',
   'coordinates': [424.4, 456.09, 407.1, 407.1],
   'attributes': ['Truncation Fully Visible', 'Occlusion Bottom & Width']}},
 1: {'box1': {'label': 'Automobile',
   'coordinates': [886.89, 476.23, 694.45, 694.45],
   'attributes': ['Truncation Right', 'Occlusion Width']}},
 2: {'box1': {'label': 'Heavy Truck',
   'coordinates': [1277.94, 585.9, 655.0, 655.0],
   'attributes': ['Truncation Left', 'Occlusion Fully Visible']}},
 3: {'box1': {'label': 'Person',
   'coordinates': [993.0, 395.92, 957.2, 957.2],
   'attributes': ['Truncation Right', 'Occlusion Width']},
  'box2': {'label': 'Automobile',
   'coordinates': [962.14, 598.23, 550.1, 

In [28]:
def get_image_list(file):
    
    soup = bs4.BeautifulSoup(file)
    
    all_id = [idd.text for idd in soup.select('segments > segment > id')]
    all_start = [int(start.text) for start in soup.select('segments > segment > start')]
    all_stop = [int(stop.text) for stop in soup.select('segments > segment > stop')]
    
    return all_id, all_start, all_stop, soup

In [29]:
def image_evaluation(image):
    
    image_dict = {}
    box_count = 0
    
    for box in image.find_all('box'):
        
        box_dict = {}
        
        box_cord = []
        
        box_dict['label'] = box.get('label')
        
        box_cord.append(float(box.get('xbr'))) # x1
        box_cord.append(float(box.get('ybr'))) # y1
        box_cord.append(float(box.get('xtl'))) # x2
        box_cord.append(float(box.get('xtl'))) # y2
        
        attribute_list = []
        
        for attribute in box.find_all('attribute'):
            
            attribute_list.append(attribute.get('name')+' '+attribute.text )
            
            
        box_dict['coordinates'] = box_cord
        box_dict['attributes'] = attribute_list
        
        box_count += 1
        
        image_dict['box'+str(box_count)] = box_dict
        
    return image_dict

### Evaluation

In [30]:
def evaluation_score(file, goal_data_dict):
    
    all_id, all_start, all_stop, soup = get_image_list(file)

    score_dict = {}

    for id_index in range(len(all_id)):

        # From image
        start = all_start[id_index]

        # Upto image
        stop = all_stop[id_index]

        image_score_dict = {}

        for img in range(start, stop+1):

            # Image content
            image = soup.find('image', id=img)

            # Image score
            image_score = 0.0
            
            # Box score
            box_score = 0.0

            # If image is annotated
            if image:

                image_dict = image_evaluation(image)


                # IMAGE EVALUATION

                # Get goal image
                goal_image = goal_data_dict[img]

                # Get attribute score for each box
                for box in image_dict.keys():

                    box1 = image_dict[box]['coordinates']
                    box2 = goal_image[box]['coordinates']
                    
                    box_score += box_iou(box1, box2)
                    
                    
                    
                    # Label is correctly selected
                    if image_dict[box]['label'] == goal_image[box]['label']:

                        correct_attr = 0

                        if image_dict[box]['attributes']:

                            # Compare each attribute
                            for attr_index in range(len(image_dict[box]['attributes'])):

                                # Attribute correctly selected
                                image_dict[box]['attributes'][attr_index] == goal_image[box]['attributes'][attr_index]

                                correct_attr += 1

                            # add correct attributes ratio
                            image_score += correct_attr/len(goal_image[box]['attributes'])

                        else:
                            correct_attr += 1
                            image_score += correct_attr

                # round off image score upto 4 decimal places            
                image_score = round(image_score / len(image_dict.keys()), 4)
                
                # round off image score upto 4 decimal places            
                box_score = round(box_score / len(image_dict.keys()), 4)

            else:

                image_score = 0.0

            image_score_dict[img] = [image_score, box_score]

        score_dict[all_id[id_index]] = image_score_dict

    return score_dict

In [31]:
file1 = open('../Downloads/Bounding_Box/git_annotation_2020_01_23_09_43_20.dump', 'r')

In [32]:
user_dict = evaluation_score(file1, goal_data_dict)

In [33]:
def box_iou(box1, box2):
    
    #Get the coordinates of bounding boxes
    b1_x2, b1_y2, b1_x1, b1_y1 = box1[0], box1[1], box1[2], box1[3]
    b2_x2, b2_y2, b2_x1, b2_y1 = box2[0], box2[1], box2[2], box2[3]
    
    I_x1 = max(b1_x1, b2_x1)
    I_y1 = max(b1_y1, b2_y1)
    I_x2 = min(b1_x2, b2_x2)
    I_y2 = min(b1_y2, b2_y2)
    
    #print(I_x1, I_y1, I_x2, I_y2)
    
    I_width = abs(I_x1 - I_x2)
    I_length = abs(I_y1 - I_y2)
    #print(I_width, I_length)
    
    # Intersection area
    I_area = I_length * I_width
    
    # Box1 area : width * length
    b1_area = abs(b1_x2 - b1_x1) * abs(b1_y2 - b1_y1)
    # Box2 area : width * length
    b2_area = abs(b2_x2 - b2_x1) * abs(b2_y2 - b2_y1)
    
    # Union area
    U_area = b1_area + b2_area - I_area
    
    #print(I_area, U_area)
    
    IoU = I_area / (U_area + 1)
    
    return round(IoU, 4)

In [34]:
box_iou(box1, box2)

NameError: name 'box1' is not defined

In [36]:
user_dict['284'][12][1]

0.0